# 컴포넌트 최적화
- 렌더링 속도를 개선하고, 불필요한 리렌더링을 줄이는 것


<br>

<hr>

<br>

## 01. 컴포넌트 최적화

<br>

### 성능 최적화
- React 애플리케이션의 성능 최적화는 필수

<br>

#### 1. 일반적인 웹 최적화
- 이미지 최적화
- HTML, CSS, JavaScript 자원 압축
- 웹 브라우저 캐싱 전략
- 지연 로딩
- HTTP/2 프로토콜 사용
  
  $\rightarrow$ **웹 브라우저와 네트워크 환경에 중점을 둔 보편적인 전략**

<br>

#### 2. React 특화 최적화
-  **React의 구조와 렌더링 방식에 대한 이해를 바탕**
   -  React는 컴포넌트 단위로 UI를 구성하고, 상태 변화에 따라 컴포넌트를 리렌더링
   
   $\rightarrow$ 컴포넌트 렌더링을 줄이고, 상태 업데이트를 효율적으로 관리하며, 불필요한 연산을 최소화

   - `React.memo` : 불필요한 리렌더링 방지
   - `React.lazy` : 필요한 시점에 컴포넌트를 불러오는 코드 스플리팅
   - `useMemo`, `useCallback` : 복잡한 연산 결과나 함수를 메모이제이션
   - `useTransition`, `useDeferredvalue` : 상태 업데이트를 비동기적으로 지연 처리

<br>

### 불필요한 리렌더링 
- 컴포넌트는 부모-자식 관계를 맺으며 컴포넌트 트리 구조로 연결
  - `App` 컴포넌트가 루트 컴포넌트일 때, `A`컴포넌트를 렌더링하고, `A`는 `B`를 렌더링, `B`는 `C`를 렌더링한다면 컴포넌트 트리는

<img src='img/1101.png' width=100>

<br>

- **상위 컴포넌트가 리렌더링되면, 그에 연결된 모든 하위 컴포넌트도 함께 리렌더링**
  
  $\rightarrow$ 컴포넌트의 상태나 `props`가 전혀 바뀌지 않았음에도 컴포넌트가 리렌더링 
  
  $\rightarrow$ **불필요한 리렌더링**

  $\rightarrow$ 컴포넌트의 수가 많아질수록 렌더링 비용도 커질 수 있으며, 성능 저하의 원인이 될 수도 있음

<br>

### 메모이제이션 (Memoization)
- **이미 계산한 결과를 저장해두고 같은 계산을 반복해야 할 때, 저장된 값을 다시 사용하는 최적화 기법**
  - 재귀 함수 등 같은 계산이 반복되는 알고리즘에서 효과적

<br>

- **피보나치 수열 계산 함수**
  - **$n$이 1이하일 때까지 재귀적으로 자신을 호출해 피보나치 수를 계산**

```ts
function fibonacci(n) {
   if (n <= 1) {
      return n;
   }
   return fibonacci(n - 1) + fibonacci(n - 2);
}

console.log(fibonacci(50));
```

<br>

- **메모이제이션을 적용한 피보나치 함수**
   - `memo`라는 객체에 이미 계산한 값을 저장
   - 같은 입력이 다시 들어오면 새로운 계산 없이 `memo`에서 값을 가져오기 때문에 중복 계산을 피할 수 있어 성능이 크게 향상

<br>

```ts
function fibonacci(n, memo = {}) {
   if (n <= 1) {
      return n;
   }

   if (meno[n]) {
      return memo[n];
   }

   memo[n] = fibonacci(n - 1 , memo) + fibonacci(n - 2, memo);

   return memo[n];
}

console.log(fibonacci(40));
```

<br>

<hr>

<br>

## 02. 컴포넌트 메모이제이션
- 컴포넌트 자체를 메모이제이션하면, 해당 컴포넌트의 `props`나 상태가 변경되지 않는 한 리렌더링 되지 않고, 이전에 렌더링한 결과를 그대로 재사용
  
  $\rightarrow$ 복잡한 연산을 수행하지 않더라도 자주 렌더링 되는 컴포넌트 렌더링 횟수를 줄이는 데 유용

<br>

### `React.memo` 
- 컴포넌트를 메모이제이션할 때는 고차 컴포넌트인 `React.memo`를 사용
  - **고차 컴포넌트 (Higher Order Component)** : **컴포넌트를 인자로 받아 새로운 컴포넌트를 반환하는 함수**
- 반환된 컴포넌트는 `props`가 변경되지 않는 한 리렌더링 되지 않고, 이전 결과를 재사용

<br>

```ts
const MemoizedComponent = React.memo(MyComponent);
```

<br>

#### `React.memo` 적용

- `components/A.tsx`

```ts
import { memo } from "react";
import B from "./B";

export default memo(function A() {
    console.log('A render');
    return (
        <>
            <h1>A Component</h1>
            <B />
        </>
    )
});
```

<br>

#### `React.memo` 적용 주의사항

<br>

- **메모이제이션의 적절한 사용**
  - `React.memo`자체도 하나의 연산이기에 불필요하게 사용하면 오히려 성능 비용이 증가
  - 기본적으로 React는 상위 컴포넌트가 리렌더링되면 하위 컴포넌트도 함께 리렌더링
  
  $\rightarrow$ '`A` -> `B` -> `C`' 구조에서는 `A`만 `React.memo`만 적용하는 것이 효율적

<br>

- **메모이제이션의 무효화 조건**
  - **메모이제이션된 컴포넌트의 자체 상태가 변경된 경우**
  - **메모이제이션된 컴포넌트로 전달되는 `props` 값이 변경된 경우**
  
<br>

<hr>

<br>

## 03. 함수 메모이제이션

<br>

### 함수를 `props`로 전달하는 경우
- 자바스크립트에서 함수, 배열, 객체는 모두 **참조 자료형 (Reference Type)**
- **참조 자료형은 변수에 값 자체가 저장되는 것이 아니라, 값이 저장된 메모리 주소(참조값)가 저장**
  
  $\rightarrow$ **같은 로직의 함수라도 다시 정의하면 참조 값이 달라지게 됨**

- 예) `App.tsx`
  - **`App`컴포넌트가 렌더링될 때마다 `increment`함수는 새롭게 생성되며 참조 값이 달라짐**
  
  $\rightarrow$ **`React.memo`는 참조 값이 달라지면 `props` 값이 변경된 것으로 판단**

  $\rightarrow$ **메모이제이션 효과를 받지 못하고 리렌더링**

```ts
export default function App() {

    const increment = () => setCount((count) => count + 1);

    ...


}
```

<br>

### `useCallback` 훅
- **컴포넌트에 전달되는 `props` 중에는 실제로 화면 렌더링에 영향을 주지 않는 함수형 `props`가 존재**
  - 예) 이벤트 핸들러나 콜백 함수처럼 UI보다 동작에 관여하는 `props`
  
  $\rightarrow$ **매번 새로운 함수 객체가 리렌더링**

<br>

#### `useCallback` 훅 사용
- **함수의 참조 값을 고정**
    - **`fn` : 메모이제이션할 함수**
    - **`dependencies` : 의존성 배열**
      - 배열의 값이 변경되지 않는 한 함수는 새로 생성되지 않고 기존 것을 재사용
      - **의존성 배열을 빈 배열(`[]`)로 설정하면 컴포넌트가 처음 렌더링될 때 한 번만 함수가 생성되고, 리렌더링 때도 동일한 참조 값의 함수가 유지**

<br>

```ts
const cachedFn = useCallback(fn, dependencies);
```

<br>

- `App.tsx`

<br>

```ts
import { useState, useCallback } from "react";
import A from "./components/A";

export default function App() {
  console.log('App render');
  const [count, setCount] = useState(0);

  // useCallback 적용
  const increment = useCallback(() => setCount((count) => count + 1), []);
  
  return (
    <>
      <h1>App Count: {count}</h1>
      <A increment={increment} />
    </>
  )
}
```

<br>

#### `useCallback` 적용 주의사항
- **상태 변경 함수를 콜백 함수 형태로 사용**
  - **`useState`훅으로 정의한 상태를 변경할 때 이전 상태값을 기준으로 새 값을 계산해야 하는 경우에는, 상태 변경함수를 콜백 함수 형태로 사용하는 것이 안전**

<br>

- **의존성 배열에 상태 값을 포함하지 않기**

<br>

<hr>

<br>

## 04. 값 메모이제이션

<br>

### 연산 비용이 큰 작업의 성능 저하 문제 
- 컴포넌트 내부에 연산 비용이 큰 코드가 있으면, 리렌더링이 발생할 때마다 성능 저하가 누적되어 사용자 경험에 악 영향을 미칠 수 있음

<br>

### `useMemo` 훅
- `useCallback`은 함수 자체를 메모이제이션하는 반면, **`useMemo`는 함수의 반환 값을 메모이제이션**
    - **`calculateValue` : 메모이제이션할 값을 계산하는 함수** 
      - 값을 직접 전달하는 것이 아닌, 이 함수의 반환 값을 메모이제이션
    - **`dependencies` : 의존성 배열**
      - 배열에 포함된 값 중 하나라도 변경되면 `calculateValue` 함수가 다시 실행되어 새로운 값을 계산하고 저장
      - 배열의 값이 변경되지 않으면 기존에 저장된 값을 그대로 재사용

<br>

```ts
const cachedValue = useMemo(calculateValue, dependencies);
```

<br>

- `src/lib/utils.ts`

```ts
export const initialItems = new Array(29_999).fill(0).map((_, i) => {
    return {
        id: i,
        selected: i === 29_999_998
    };
});
```

<br>

- `App.tsx`
  - `useMemo` 훅을 사용하여 `selectedItem`을 메모이제이션
    - 의존성 배열이 빈 배열 `[]`이기에, 최초 렌더링 때 한 번만 계산되고, 리렌더링 될 때는 저장된 결과를 그대로 재사용

    $\rightarrow$ 애플리케이션이 종료될 때 까지 `initialItems.find(...)`는 다시 실행되ㅣㅈ 않음 

```ts
export default function App() {
  console.log('App render');
  const [count, setCount] = useState(0);

  const selectItems = useMemo(() => initialItems.find((item) => item.selected), []);
  
  return (
    <>
      <h1>Count: {count}</h1>
      <button onClick={() => setCount((prevCount) => prevCount + 1)}>증가</button>
      <p>{selectItems?.id}</p>
    </>
  )
}
```

<br>

#### `useMemo` 적용 주의사항
- **무분별한 메모이제이션 사용 지양**
- **잘못된 의존성 배열**
  - 의존성 배열에 불필요한 값을 넣으면, 해당 값이 변경될 때마다 메모이제이션된 값을 다시 계산하게 되어, 결국 `useMemo` 훅을 쓰지 않은 것과 차이가 없어짐
    

```ts
export default function App() {

  ...

  // count를 의존성 배열에 넣어, count가 변경될 때마다, initialItems.find(...)연산이 매번 다시 실행
  const selectItems = useMemo(
    () => initialItems.find((item) => item.selected), [count]
  );

  return (...);
}
```

<br>

- **연산 비용이 낮은 값에 대한 메모이제이션 지양**
- **메모이제이션할 때 `useMemo` 훅으로 함수 감싸지 않기**

<br>

- **기능적으로는 `useCallback`훅과 `useMemo` 훅이 거의 동일하지만, 함수 메모이제이션에는 `useCallback` 훅을 사용하는 것이 더 적절하고 명확**
  - **의도의 명확성**
    - `useCallback` 훅은 함수를, `useMemo` 훅은 값을 메모이제이션하는 것이 목적
  - **코드 명확성**
    - `useCallback` 훅을 사용하면 함수 중첩 없이 더 간결하게 코드 작성 가능

<br>

<hr>

<br>



## 05. 로딩 성능 최적화

<br>

### `React.lazy()`를 사용한 코드 스플리팅
- **코드 스플리팅 (Code Splitting) : 애플리케이션의 코드를 청크 단위로 분할하고, 필요한 시점에만 해당 청크를 로드해 초기 렌더링 시 불필요한 코드의 로드를 지연시키는 최적화 기법**
  
    $\rightarrow$ 사용자가 특정 기능이나 화면을 요청하기 전까지는 그에 해당하는 컴포넌트를 로드하지 않기에, 초기 로딩 속도를 줄이고 체감 성능을 높임

<br>

- **코드 스플리팅 적용 이전**
  - **`isShow` 상태가 `true`일 때만 `LazyComponent`를 조건부로 렌더링**
  
  $\rightarrow$ **하지만 초깃값이 `false`인데도 `LazyComponent`는 애플리케이션이 처음 로드될 때 불러와짐**

  **([개발자 도구] $\rightarrow$ [Network 탭]에서 `LazyComponent.tsx`가 초기 페이지 로드 시점에 함께 로드됨을 확인 가능)**

```ts
import { useState } from "react";
import LazyComponent from "./components/LazyComponent";

export default function App() {
  const [isShow, setIsShow] = useState(false);

  return (
    <>
      <button onClick={() => setIsShow(!isShow)}>Toggle</button>
      {isShow && <LazyComponent />}
    </>
  );
}
```

<br>

- **코드 스플리팅 적용**
    - **`LazyComponent`를 필요한 시점에만 로드**

```ts
import { lazy, useState } from "react";
const LazyComponent = lazy(() => import('./components/LazyComponent'));

export default function App() {
  const [isShow, setIsShow] = useState(false);

  return (
    <>
      <button onClick={() => setIsShow(!isShow)}>Toggle</button>
      {isShow && <LazyComponent />}
    </>
  );
}
```

<br>

#### `React.lazy()` 사용법
- React에서 컴포넌트를 지연 로딩 (lazy loading)할 때 사용하는 메서드

```ts
import { lazy } from 'react';
const LazyComponent = lazy(() => import('./components/LazyComponent'));
```

<br>

### `Suspense`
- **Suspense는 React 16.6에서 도입된 내장 컴포넌트로, 비동기 작업이 완료될 때까지 렌더링을 일시 중지하는 기능을 제공**
  - 주료 코드 스플리팅이나 데이터 패칭처럼 지연 로딩할 때 활용
  - 데이터 패칭 (Data Fetching) : 애플리케이션이 외부 API나 DB에서 필요한 요청을 받아오고 하는 작업

<br>

#### `Suspense` 사용
- **`Suspense`는 비동기 로딩 완료될 때까지 `fallback` 속성으로 지정한 UI를 대신 보여주는 역할**

<br>

- `App.tsx`
  - **버튼을 누르면 2초동안의 딜레이 후, `LazyComponent`가 로드**
  - **딜레이 기간동안 `<Suspense>`의 `fallback`인자의 내용이 표시**

```ts
import { lazy, Suspense, useState } from "react";

// 파일을 import 했을 때 그 파일의 default export 타입만 로드
type LazyModuleDefault = typeof import("./components/LazyComponent").default;

const LazyComponent = lazy<LazyModuleDefault>(
  () => new Promise<{ default: LazyModuleDefault }>(
    (resolve) => {
      setTimeout(
        () => {
          import("./components/LazyComponent").then(
            (mod) => {
              resolve(mod);
          });
        }, 2000);
    })
);

export default function App() {
  const [isShow, setIsShow] = useState(false);
  return (
    <>
      <button onClick={() => setIsShow((s) => !s)}>Toggle</button>
      {isShow && (
        <Suspense fallback={<div>Loading...</div>}>
          <LazyComponent />
        </Suspense>
      )}
    </>
  );
}
```

<br>

#### `Suspense` 구성 방식
- `Suspense`는 컴포넌ㅌ느를 하나만 감쌀 수도 있고, 여러 컴포넌트를 묶어서 그룹화하거나 각 컴포넌트를 개별로 나눠서 감쌀 수도 있음
- `fallback` 속성에는 JSX 요소뿐 아니라, 별도의 로딩 컴포넌트도 지정할 수 있음

<br>

- **`Suspense`를 두 그룹으로 나누어 적용**
  - **두 그룹이 병렬로 비동기 로드되기 때문에, 먼저 로드가 완료된 그룹은 바로 렌더링되고, 아직 로딩 중인 그룹은 `Loading` 컴포넌트가 대신 표시**

```ts
export default function App() {
  
  ...

  return (
    <>

      ...

      {isShow && (
        <>
          <Suspense fallback={<Loading />}>
            <LazyComponent1 />
            <LazyComponent2 />
          </Suspense> 

          <Suspense fallback={<Loading />}>
            <LazyComponent3 />
            <LazyComponent4 />
          </Suspense> 
        </>
      )}

    </>
  )
}
```

<br>

### `ErrorBoundary`
- `ErrorBoundary`는 React 16부터 도입된 오류 복구용 컴포넌트 패턴
- **컴포넌트 트리 내부에서 자바스크립트 오류가 발생해도 애플리케이션 전체가 멈추지 않도록 보호** 
  
  $\rightarrow$ **오류가 발생한 컴포넌트를 감싸서 대체 UI를 보여주는 방식으로 사용자 경험을 유지할 수 있음**

<br>

#### `react-error-boundary`
- React에서 `ErrorBoundary` 패턴을 쉽게 구현할 수 있는 라이브러리


```bash
$ npm install react-error-boundary
```

<br>

- **`ErrorBoundary`에서는 오류 발생 시 보여줄 UI를 두 가지 방식으로 지정 가능**
  - **`fallback` 속성 : JSX 요소를 직접 전달**
  - **`FallbackComponent` 속성 : 별도의 컴포넌트를 만들어 전달**

<br>

#### `fallback` 속성

```ts
import { ErrorBoundary } from "react-error-boundary";

...

export default function App() {
  const [isShow, setIsShow] = useState(false);
  return (
    <>
      <button onClick={() => setIsShow((s) => !s)}>Toggle</button>
      {isShow && (
        <ErrorBoundary fallback={<div>오류 발생</div>}>
          <Suspense fallback={<div>Loading...</div>}>
            <LazyComponent />
          </Suspense>
        </ErrorBoundary>
      )}
    </>
  );
}
```

<br>

#### `FallbackComponent` 속성 

- `App.tsx`
  - 오류가 발생했을때 발생하는 `props`를 `Fallback` 컴포넌트에 전달

```ts
import { ErrorBoundary } from "react-error-boundary";
import Fallback from './components/Fallback';

...

export default function App() {
  const [isShow, setIsShow] = useState(false);
  return (
    <>
      <button onClick={() => setIsShow((s) => !s)}>Toggle</button>
      {isShow && (
        <ErrorBoundary FallbackComponent={Fallback}>
          <Suspense fallback={<div>Loading...</div>}>
            <LazyComponent />
          </Suspense>
        </ErrorBoundary>
      )}
    </>
  );
}
```

<br>

- `components/Fallback.tsx`
  - **전달받은 `props`**
    - **`error` : 실제 발생한 오류객체**
    - **`resetErrorBoundary()` : 오류상태를 초기화하고 오류가 난 컴포넌트를 리렌더링하는 함수**

```ts
export default function Fallback({
  error, resetErrorBoundary()
} : {
  error: Error;
  resetBoundary: () => void;
}) {
  return (
    <div role='alert'>
      <p>오류 발생:</p>
      <pre style={{ color: 'red' }}>{error.message}</pre>
      <button onClick={resetErrorBoundary}>retry</button>
    </div>
  )
}
```

<br>

#### React 19에서 개선한 오류 처리 방식
- React 19에서는 오류 처리 방식이 개선
- **오류 상황에 따라 대응할 수 있도록 전용 콜백함수 제공**
  - **`onCaughtError`** : `ErrorBoundary`에 잡힌 오류가 발생하면 호출
  - **`onUncaughtError`** : `ErrorBoundary`에 잡히지 않은 오류가 발생하면 호출
  - **`onRecoverableError`** : `ErrorBoundary`로 오류를 처리한 후 [retry]버튼 등을 클릭해 컴포넌트를 리렌더링할 때 호출

<br>

<hr>

<br>

## 06. 상태 업데이트 최적화

<br>

### `useDeferredValue` 훅
- 사용자가 입력칸에 글자를 입력할 때마다 화면이 무거워지고 버벅거린다면, 입력 값이 바뀔 때마다 화면 전체를 리렌더링하는 것이 원인일 수 있음
  
  $\rightarrow$ **입력값은 즉시 업데이트하되, 그에 따른 UI 반영은 조금 늦추면 성능 개선 가능**

  $\rightarrow$ **`useDeferredValue` 훅**

- 초깃값을 지정하면 초기 렌더링 시 해당 값을 먼저 사용해 화면을 표시하고, 이후 백그라운드에서 `value`값을 기반으로 업데이트를 실행

```ts
const deferredValue = useDeferredValue(value, 초깃값);
```

<br>

- `App.tsx`

```ts
import { useDeferredValue, useState } from "react";
import SlowList from "./components/SlowList";

export default function App() {
  const [query, setQuery] = useState("");
  return (
    <div>
      <input
        type="text"
        value={query}
        onChange={(e) => setQuery(e.target.value)}
      />
      <SlowList query={query} />
    </div>
  );
}
```

<br>

- `components/SlowList.tsx`
  - 전달받은 `query` 값을 기반으로 500개 항목을 렌더링
  - 각 항목은 1밀리초 동안 인위적인 지연 작업을 수행하므로, 전체 리스트를 렌더링하는데 약 500밀리초가 소요
  

```ts
import React from "react";

export default React.memo(function SlowList({ query }: { query: string }) {
  console.log("slowlist");
  const items = [];
  // 1ms 작업 지연을 500번 반복하므로 총 500ms 소요
  for (let i = 0; i < 500; i++) {
    items.push(<SlowItem key={i} query={query} />);
  }
  return <ul>{items}</ul>;
});

function SlowItem({ query }: { query: string }) {
  const startTime = performance.now();
  while (performance.now() - startTime < 1) {
    /* empty */
  } // 1ms 동안 작업 지연
  return <li>query: {query}</li>;
}
```

<br>

- **입력칸에 글자를 입력할 때마다 화면 전체가 멈추는 듯한 지연 현상 발생**
  
  - 예) 사용자가 `123`을 입력하면, `query`값은 총 3번 바뀌고, 그떄마다 `SlowList` 컴포넌트를 다시 그리게 됨

  $\rightarrow$ **`useDeferredValue` 훅을 사용하여, `query`값 전달을 지연시켜, 사용자가 빠르게 입력하는 도중에는 렌더링을 건너뛰고 최종 입력 값에 따라 한 번만 렌더링 되도록 함**

<br>

- `App.tsx`
  - **`deferredValue`는 렌더링에 부담을 주지 않는 범위 내에서 가능한 늦게 업데이트 되도록 React가 자동으로 조절**
  
  $\rightarrow$ **사용자가 빠르게 입력 값을 바꾸더라도 렌더링은 그 중 일부만 선택적으로 수행**

```ts
import { useDeferredValue, useState } from "react";
import SlowList from "./components/SlowList";

export default function App() {
  const [query, setQuery] = useState("");
  const deferredValue = useDeferredValue(query);
  return (
    <div>
      <input
        type="text"
        value={query}
        onChange={(e) => setQuery(e.target.value)}
      />
      <SlowList query={deferredValue} />
    </div>
  );
}
```

<br>

- **`useDeferredValue`훅은 `React.memo`와 함꼐 사용하는 것이 좋음**
- **`useDeferredValue`훅은 상태 값의 업데이트 시점을 지연시지만, 컴포넌트의 리렌더링 자체는 막지 못함**
  
  $\rightarrow$ **`React.memo`로 감싸면 전달된 `deferredValue` 값이 이전 렌더링과 동일한 경우에는 리렌더링을 건너뜀**

  $\rightarrow$ **`React.memo`는 `useDeferredValue`와 함께 사용할 때 불필요한 렌더링을 방지하는 데 중요한 역할**

<br>

### `useTransition` 훅
- React에서 UI의 일부 렌더링 작업을 백그라운드에서 처리할 수 있도록 함
  
  $\rightarrow$ **사용자의 입력과 같은 상호작용은 즉시처리하면서도, 무거운 UI 업데이트는 지연시켜 더 자연스럽고 부드러운 사용자 경험을 제공**

<br>

- **`isPending` : 현재 백그라운드 작업이 진행 중인지 여부를 나타냄**
  - `true`일 경우 작업이 진행중이며, `false`일 경우 작업이 완료된 상태
- **`startTransition` : 지정한 작업을 백그라운드에서 지연 실행**

```ts
const [isPending, startTransition] = useTransition();
```

<br>

- `App.tsx`
  - 사용자 입력에 따라 작업이 진행중이면 (`isPending === true`), 로딩 컴포넌트 표시,
  
    작업이 종료되면 결과 표시

```ts
import { useState, useTransition } from "react";
import SlowList from "./components/SlowList";

export default function App() {
  const [query, setQuery] = useState("");
  const [defferedValue, setDeferredValue] = useState("");
  const [isPending, startTransition] = useTransition();

  const handleChange = (e: React.ChangeEvent<HTMLInputElement>) => {
    const newQuery = e.target.value;
    setQuery(newQuery);
    startTransition(() => setDeferredValue(newQuery));
  };

  return (
    <div>
      <input type="text" value={query} onChange={handleChange} />
      {isPending ? <div>Loading</div> : <SlowList query={defferedValue} />}
    </div>
  );
}
```

<br>

<hr>

<br>

## 07. 리소스 로딩 최적화 (React 19 이후)

- React 19에서는 로딩 지연 문제를 해결하기 위해 웹 브라우저가 필요한 리소스를 미리 불러올 수 있도록 도와주는 API들을 새롭게 제공
  - 이 API들은 네트워크 지연을 줄이고, 페이지 렌더링을 빠르게 만들어주는 데 효과적    

| **API** | **설명** |
| - | - |
| `preconnect()`  | 아직 어떤 리소스를 요청할지 명확하지 않더라도 서버와의 연결을 미리 설정  |
| `prefetchDNS()`  |  요청할 가능성이 있는 도메인의 IP주로를 미리 조회(DNS Lookup)해 지연을 줄임 |
| `preinit()`  | 외부 스크립트나 스타일시트를 미리 로드하고 실행하거나 삽입  |
| `preinitModule()`  | ESM(ECMAScript Module)을 미리 로드해 실행  |
| `preload()`  |  스타일시트, 폰트, 이미지, 외부 스크립트 등 필요한 리소스를 미리 불러옴 |
| `preloadModule()`  |  사용할 가능성이 있는 ESM을 미리 불러옴 |

<br>

- 사용 예시

```ts
import { prefetchDNS, preconnect, preload, preinit } from 'react-dom';

function MyComponent() {
    // 외부 스크립트 미리 실행
    preinit('https://.../path/to/some/script.js', { as: 'script' });

    // 웹 폰트 미리 로드
    preload('https://.../path/to/font.woff', { as: 'font' });

    // 스타일시트 미리 로드
    preload('https://.../path/to/stylesheet.css', { as: 'style' });

    // DNS 조회 사전 수행
    prefetchDNS('https://...');

    // 서버와 사전 연결 설정
    preconnect('https://...');
}
```

<br>

- **이 컴포넌트를 실행하면 React는 다음과 같은 `<head>`태그 내부의 HTML 코드를 자동으로 생성**

```html
<html>
  <head>
      <link rel='prefetch-dns' href='https://...'>
      <link rel='preconnect' href='https://...'>
      <link rel='preload' as="font" href='https://.../path/to/font.woff'>
      <link rel='preload' as="style" href='https://.../path/to/stylesheet.css'>
      <script async="" src="https://.../path/to/some/scripts.js"></script>
  </head>
  <body>
    
    ...

  </body>

</html>
```

<br>

<hr>

<br>


## 08. 할 일 관리 애플리케이션 개선

<br>

### 불필요한 리렌더링 코드

- 할 일을 1개 등록하면 로그가 1개 출력
  
  할 일을 2개 등록하면 로그가 2개 출력

  $\rightarrow$ **할 일을 추가할수록 콘솔에 출력되는 로그 수도 함께 증가**

  $\rightarrow$ **할 일을 추가할 때 기존 항목들까지 모두 리렌더링 되고 있다는 의미**

<img src='img/1102.png' width=500>

<br>

### 불필요한 리렌더링 최적화
- 할 일을 추가할 때마다 `TodoListItem` 컴포넌트가 불필요하게 리렌더링
  
<br>

#### 1. `React.memo` 훅 적용
- `TodoListItem` 컴포넌트를 메모이제이션하여 해당 항목의 `props`가 변경되지 않는 한 리렌더링이 발생하지 않게 됨
- 

- `components/TodoListItem.tsx`

```ts
import { useState, memo } from "react";

...

export default memo(function TodoListItem({

  ...

}))
```

<br>

#### 2. `useCallback` 훅 적용
- `React.memo`를 적용하더라도 함수형 `props`의 참조값이 변경되면 컴포넌트가 다시 리렌더링
  
  $\rightarrow$ **`toggleTodo`, `deleteTodo`, `modifyTodo` 함수를 `useCallback` 훅으로 감싸 참조값이 변하지 않도록 메모이제이션**

<br>

- `App.tsx`

```ts
import { useState, useEffect, useCallback } from 'react';

...

export default function App() {

  ...

  const toggleTodo = useCallback((id:number) => {
    setTodos(
      (todos) => todos.map(
        (todo) => todo.id === id ? { ...todo, done: !todo.done } : todo
      )
    );
  }, []);

  const deleteTodo = useCallback((id: number) => {
    setTodos(
        (todos) => todos.filter(
            (todo) => todo.id !== id
        )
    );
  }, []);

  const modifyTodo = useCallback((id: number, title: string) => {
    setTodos(
      (todos) => todos.map(
        (todo) => (todo.id === id ? {...todo, title } : todo)
      )
    )
  }, []);

  ...

  return (

    ... 

  )
}
```

<br>

<hr>

<br>